In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Colab에서 작업
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 원본 데이터 읽기
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csvs/winemag-data-130k-v2.csv')

In [5]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# 0. EDA 전 전처리
1) 완전히 동일한 행 삭제 <br>
2) 1)에서 삭제한 df로 결측치 비율 확인 <br>
3) taster_twitter_handle 삭제

## 1) 완전히 동일한 행 삭제 

In [6]:
# Unnamed: 0 칼럼 삭제
df.drop(columns='Unnamed: 0', inplace=True)

In [7]:
duplicates = df[df.duplicated()]

In [8]:
duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9983 entries, 2408 to 129913
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                9979 non-null   object 
 1   description            9983 non-null   object 
 2   designation            7063 non-null   object 
 3   points                 9983 non-null   int64  
 4   price                  9382 non-null   float64
 5   province               9979 non-null   object 
 6   region_1               8296 non-null   object 
 7   region_2               3742 non-null   object 
 8   taster_name            8656 non-null   object 
 9   taster_twitter_handle  8216 non-null   object 
 10  title                  9983 non-null   object 
 11  variety                9983 non-null   object 
 12  winery                 9983 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.1+ MB


In [9]:
dups = df.groupby(by=df.columns.tolist()).size().to_frame('dupls')
set(dups.dupls) # 중복된 행들은 모두 2회씩만 중복. 

{1, 2}

In [10]:
df = df.drop_duplicates()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119988 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                119929 non-null  object 
 1   description            119988 non-null  object 
 2   designation            85443 non-null   object 
 3   points                 119988 non-null  int64  
 4   price                  111593 non-null  float64
 5   province               119929 non-null  object 
 6   region_1               100428 non-null  object 
 7   region_2               46769 non-null   object 
 8   taster_name            95071 non-null   object 
 9   taster_twitter_handle  90542 non-null   object 
 10  title                  119988 non-null  object 
 11  variety                119987 non-null  object 
 12  winery                 119988 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.8+ MB


> 중복 행 제거 후 데이터프레임은 129,971행의 최초 데이터프레임에서 9983행(=중복된 행들은 각각 2번씩만 중복됐으므로 중복 행 수와 동일)이 제거된 119,888*13 사이즈

> 

## 2) 칼럼별 결측치 비율 확인

In [12]:
# 칼럼별 결측치 개수
df.isnull().sum().to_frame('nan_cnt')

,nan_cnt
country,59
description,0
designation,34545
points,0
price,8395
province,59
region_1,19560
region_2,73219
taster_name,24917
taster_twitter_handle,29446


In [13]:
nanratio = pd.DataFrame(data = 100*df.isnull().sum()/len(df),
                        columns=['nan_ratio(%)'])
nanratio

,nan_ratio(%)
country,0.049172
description,0.000000
designation,28.790379
points,0.000000
price,6.996533
province,0.049172
region_1,16.301630
region_2,61.021936
taster_name,20.766243
taster_twitter_handle,24.540787


- region_2 결측비율이 61%로 매우 높음

## 3) taster_twitter_handle 칼럼 삭제
- (트위터 아이디, 평가자 명) = <br>
(nan, not nan), (nan, nan), (not nan), (not nan, not nan)인 경우 4가지 존재 가능.
> 확인 결과, taster_name이 nan값이면 taster_twitter_handle이 nan값임. <br>
> taster_twitter_handle이 결측비율이 더 높으므로 삭제해도 된다고 판단. 

In [14]:
len(df[df.taster_name.isnull()]) == len(df[(df.taster_name.isnull()) & (df.taster_twitter_handle.isnull())])

True

In [15]:
# taster_twitter_handle 삭제
df = df.drop(columns='taster_twitter_handle')

In [16]:
df.info() # 이 df로 향후 분포확인, EDA, 모델 학습 (1,2 / 3,4번 단계) 진행하면 될듯.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119988 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      119929 non-null  object 
 1   description  119988 non-null  object 
 2   designation  85443 non-null   object 
 3   points       119988 non-null  int64  
 4   price        111593 non-null  float64
 5   province     119929 non-null  object 
 6   region_1     100428 non-null  object 
 7   region_2     46769 non-null   object 
 8   taster_name  95071 non-null   object 
 9   title        119988 non-null  object 
 10  variety      119987 non-null  object 
 11  winery       119988 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 11.9+ MB


In [17]:
df.to_csv('basic_processed_df')